In [1]:
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model

In [2]:
#Load trained model , scaler,ohe,label

model = load_model('model.h5')


#load encoders and scaler
with open('label_encoder_gender.pkl','rb') as file:
    label_encode_gender = pickle.load(file)

with open('ohe_encoder_geo.pkl','rb') as file:
    geo_encoder = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

In [32]:
#Example input data

input_data = {
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [33]:
#onehot encode
geo_encode = geo_encoder.transform([[input_data['Geography']]]).toarray()
geo_encode_df = pd.DataFrame(geo_encode,columns=geo_encoder.get_feature_names_out())
geo_encode_df

s:\ANN_Churn_Prediction\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [34]:
input_data = pd.DataFrame([input_data])

In [35]:
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [36]:
input_data['Gender'] = label_encode_gender.transform(input_data['Gender'])

In [37]:
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [38]:
input_data = pd.concat([input_data.drop("Geography",axis=1),geo_encode_df],axis=1)

In [39]:
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [41]:
#scaling the input data
input_scaled = scaler.transform(input_data)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [45]:
#predict churn 
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


array([[0.02391004]], dtype=float32)

In [46]:
prediction_prob = prediction[0][0]
prediction_prob

np.float32(0.023910044)

In [47]:
if prediction_prob > 0.5:
    print('The Customer is likely to chrun')
else:
    print('The customer is not likely to chrun')

The customer is not likely to chrun
